In [8]:
# This is a complete Python code example for building a basic Retrieval-Augmented Generation (RAG) pipeline
# using Groq LLM for a healthcare AI assistant. It handles document ingestion, embedding, retrieval,
# generation, and multi-turn conversation memory.
#
# Prerequisites:
# - Install required packages: pip install groq langchain langchain-community langchain-groq sentence-transformers pypdf docx2txt openpyxl pillow pytesseract
# - Set your GROQ_API_KEY as an environment variable.
# - For image processing (OCR), install Tesseract OCR and set its path.
#
# This example assumes a simple setup: Upload documents to a folder, process them, and chat via console.
# For production, integrate with a web interface and use a robust vector store like Pinecone or FAISS.

import os
from groq import Groq
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredExcelLoader, UnstructuredImageLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain








In [10]:
# Set up Groq LLM (replace with your preferred model, e.g., 'llama-3.3-70b-versatile')
os.environ["GROQ_API_KEY"] = "Fill you Groq Api"  # Or set as env var
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)

In [11]:
# Embedding model for vectorization
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_36/1803072595.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2025-09-21 17:08:48.618201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758474528.839483      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758474528.904602      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to r

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:

# Function to load and process multi-format documents
def load_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        filepath = os.path.join(directory_path, filename)
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(filepath)
        elif filename.endswith(".docx"):
            loader = Docx2txtLoader(filepath)
        elif filename.endswith(".xlsx"):
            loader = UnstructuredExcelLoader(filepath)
        elif filename.endswith((".jpg", ".png")):  # For images with tables/charts
            loader = UnstructuredImageLoader(filepath)  # Uses OCR for text extraction
        else:
            continue
        documents.extend(loader.load())
    return documents

In [13]:
# Function to build the RAG pipeline
def build_rag_pipeline(directory_path):
    # Load documents
    docs = load_documents(directory_path)
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    # Create vector store
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    
    # Set up retriever
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    
    # Custom prompt template for healthcare context
    prompt_template = """You are a helpful medical assistant. Use the following pieces of context to answer the question accurately. 
    If you don't know the answer, say so. Maintain conversation history.

    Context: {context}
    History: {chat_history}
    Question: {question}
    Answer:"""
    PROMPT = PromptTemplate(input_variables=["context", "chat_history", "question"], template=prompt_template)
    
    # Set up memory for multi-turn conversations
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
    
    # Create conversational RAG chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )
    
    return qa_chain

In [17]:


# Example usage
if __name__ == "__main__":
    # Specify directory with uploaded documents (e.g., PDFs, Word, Excel, Images)
    doc_directory = "/kaggle/input/meidcal-mohit"  # Replace with your path
    
    # Build the pipeline
    rag_chain = build_rag_pipeline(doc_directory)
    
    # Interactive chat loop
    print("Healthcare AI Assistant ready. Type 'exit' to quit.")
    while True:
        query = input("User: ")
        if query.lower() == "exit":
            break
        result = rag_chain({"question": query})
        print("Assistant:", result["answer"])

/tmp/ipykernel_36/2322894056.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")


Healthcare AI Assistant ready. Type 'exit' to quit.


User:  tell me about patient disease


/tmp/ipykernel_36/1444166090.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain({"question": query})


Assistant: Based on the medical report, it appears that Mr. Tan Ah Kow has been diagnosed with Dementia, which is causing him to exhibit behavioural and psychological symptoms. The clinical impression is that he is manifesting symptoms secondary to Dementia. 

As for his disease history, the report mentions that Mr. Tan is at present incontinent, unable to bathe or use the toilet on his own, but is able to feed himself. There is also a mention of a gradual deterioration in his cognitive ability and physical state over the years. However, the exact details of his disease history are not fully provided in the given context.

Would you like to know more about his mental capacity assessment or something else?


User:  exit


In [16]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 75.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/

In [2]:
pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 9.4 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.26
    Uninstalling langchain-0.3.26:
      Successfully uninstalled langchain-0.3.26
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t